<a href="https://colab.research.google.com/github/vhpooya/CMS/blob/master/IranLaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch] datasets pandas accelerate

In [2]:
# اتصال به گوگل درایو برای دسترسی به فایل‌ها
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers peft bitsandbytes sentencepiece faiss-cpu PyMuPDF regex hazm

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of hazm to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.5/367.5 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
import os

# مسیر ذخیره روی Drive
drive_path = "/content/drive/MyDrive/legal_model_base"
os.makedirs(drive_path, exist_ok=True)

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ذخیره مدل و tokenizer روی Drive
tokenizer.save_pretrained(drive_path)
model.save_pretrained(drive_path)

print(f"Model and tokenizer saved to {drive_path}")


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model and tokenizer saved to /content/drive/MyDrive/legal_model_base


In [6]:
from google.colab import drive
from datasets import load_dataset
import os
import pickle

# وصل کردن Google Drive
drive.mount('/content/drive')

# مسیر ذخیره روی Drive
drive_path = "/content/drive/MyDrive/legal_dataset"
os.makedirs(drive_path, exist_ok=True)

# مسیر فایل tokenized دیتاست
tokenized_file = os.path.join(drive_path, "tokenized_dataset.pkl")

# بررسی وجود دیتاست پردازش شده
if os.path.exists(tokenized_file):
    # اگر قبلاً ذخیره شده بود، مستقیم بارگذاری کن
    with open(tokenized_file, "rb") as f:
        tokenized_dataset = pickle.load(f)
    print(f"Tokenized dataset loaded from {tokenized_file}")
else:
    # بارگذاری دیتاست از Hugging Face
    dataset = load_dataset("PerSets/iran-legal-persian-qa")

    # پیش‌پردازش دیتاست
    def preprocess(batch):
        inputs = ["سوال: " + q for q in batch["questionBody"]]
        targets = [a[0] if isinstance(a, list) and len(a) > 0 else "" for a in batch["answers"]]  # اولین جواب را بگیر
        model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Tokenize کردن دیتاست
    tokenized_dataset = dataset.map(preprocess, batched=True)

    # ذخیره tokenized dataset روی Drive با pickle
    with open(tokenized_file, "wb") as f:
        pickle.dump(tokenized_dataset, f)
    print(f"Tokenized dataset saved to {tokenized_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/600112 [00:00<?, ? examples/s]

Tokenized dataset saved to /content/drive/MyDrive/legal_dataset/tokenized_dataset.pkl


In [7]:
from peft import LoraConfig, get_peft_model, PeftModel
import os

# مسیر ذخیره LoRA روی Drive
drive_path = "/content/drive/MyDrive/legal_model_lora"
os.makedirs(drive_path, exist_ok=True)

# پیکربندی LoRA
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],  # ماتریس‌های مورد نظر برای LoRA
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

# اعمال LoRA روی مدل پایه
model = get_peft_model(model, config)

# ذخیره مدل LoRA روی Drive (checkpoint اولیه)
model.save_pretrained(drive_path)
print(f"LoRA model saved to {drive_path}")


LoRA model saved to /content/drive/MyDrive/legal_model_lora


In [14]:
# اتصال Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import pickle
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers.trainer_utils import get_last_checkpoint
from peft import LoraConfig, get_peft_model, PeftModel

# مسیرها روی Drive
drive_model_base = "/content/drive/MyDrive/legal_model_base"   # مدل پایه
drive_lora = "/content/drive/MyDrive/legal_model_lora"         # LoRA
drive_dataset = "/content/drive/MyDrive/legal_dataset"         # دیتاست tokenized
os.makedirs(drive_lora, exist_ok=True)

# بارگذاری مدل پایه و tokenizer
tokenizer = AutoTokenizer.from_pretrained(drive_model_base)
model_base = AutoModelForSeq2SeqLM.from_pretrained(drive_model_base)

# Freeze the base model parameters
for param in model_base.parameters():
    param.requires_grad = False

# اعمال LoRA یا بارگذاری از Drive
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

if os.path.exists(drive_lora) and len(os.listdir(drive_lora)) > 0:
    model = PeftModel.from_pretrained(model_base, drive_lora)
    print("LoRA model loaded from Drive")
else:
    model = get_peft_model(model_base, config)
    model.save_pretrained(drive_lora)
    print("LoRA model initialized and saved to Drive")

# Enable gradient for LoRA parameters
for name, param in model.named_parameters():
    if "lora_" in name:
        param.requires_grad = True


# Print requires_grad status for model parameters
print("\nParameters requires_grad status after applying LoRA:")
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")


# بارگذاری دیتاست tokenized از Drive
tokenized_file = os.path.join(drive_dataset, "tokenized_dataset.pkl")
with open(tokenized_file, "rb") as f:
    tokenized_dataset = pickle.load(f)
print("Tokenized dataset loaded from Drive")

# تنظیمات Trainer
training_args = TrainingArguments(
    output_dir=drive_lora,       # checkpoint ها روی Drive ذخیره شوند
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    fp16=True,
    save_strategy="epoch",
    save_total_limit=3,
    report_to="none"             # wandb غیرفعال
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    # eval_dataset=tokenized_dataset.get("test")  # اگر split test داشته باشی
)

# بررسی آخرین checkpoint و اجرای آموزش
last_checkpoint = get_last_checkpoint(training_args.output_dir)
if last_checkpoint:
    print(f"Resuming from checkpoint {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("No checkpoint found, starting training from scratch")
    trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
LoRA model loaded from Drive

Parameters requires_grad status after applying LoRA:
base_model.model.shared.weight: False
base_model.model.encoder.block.0.layer.0.SelfAttention.q.base_layer.weight: False
base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight: True
base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight: True
base_model.model.encoder.block.0.layer.0.SelfAttention.k.weight: False
base_model.model.encoder.block.0.layer.0.SelfAttention.v.base_layer.weight: False
base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight: True
base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight: True
base_model.model.encoder.block.0.layer.0.SelfAttention.o.weight: False
base_model.model.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: False
base_model.

Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000
2000,0.000000


KeyboardInterrupt: 

In [ ]:
print(tokenized_dataset.keys())

dict_keys(['train'])
